In [1]:
import torch as tr
import su2_chain as s


In [2]:
a = tr.normal(0.0,2.0,[4])
a = 2*a/tr.norm(a)
print(a)
U = s.Mat(a) #tr.sum(a[:,None,None] * basis,dim=0)
print(U)
print("Should be identity")
print(s.TdxT(U,U))

tensor([ 0.8201, -1.2608, -0.3531, -1.2701])
tensor([[ 0.4101-0.6350j,  0.1765-0.6304j],
        [-0.1765-0.6304j,  0.4101+0.6350j]])
Should be identity
tensor([[1.0000-8.2928e-09j, 0.0000+0.0000e+00j],
        [0.0000+0.0000e+00j, 1.0000+8.2928e-09j]])


In [3]:
b = tr.normal(0.0,2.0,[4])
b = 2*b/tr.norm(b)
print(b)
V = s.Mat(b) #tr.sum(a[:,None,None] * basis,dim=0)
print(V)
print("Should be identity")
print(s.TdxT(V,V))

tensor([ 0.9305, -0.7267, -1.5619,  0.4081])
tensor([[ 0.4652+0.2041j,  0.7810-0.3633j],
        [-0.7810-0.3633j,  0.4652-0.2041j]])
Should be identity
tensor([[1.0000-1.0293e-08j, 0.0000+0.0000e+00j],
        [0.0000+0.0000e+00j, 1.0000+1.0293e-08j]])


In [4]:
tr.trace(U@tr.conj(V.T)).real.item()/2-(a@b/4).item()

0.0

In [5]:
su2f = s.field([128,128],Nbatch=2)

In [6]:
su2c = s.SU2chain(beta=2.0,field_type=su2f)

In [7]:
P = su2c.refreshP()

In [8]:
U =su2c.f.hot()
U.shape

torch.Size([2, 128, 128, 4])


torch.Size([2, 128, 128, 2, 2])

In [9]:
a= su2c.f.coeffs(U)

In [10]:
a.shape

torch.Size([2, 128, 128, 4])

In [11]:
s.Mat(a[0,0,0,:])- U[0,0,0,:,:]

tensor([[0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]])

In [12]:
FF = su2c.f.mat(a)

In [13]:
(FF-U).norm()

tensor(0.)

In [14]:
P.shape

torch.Size([2, 128, 128, 2, 2])

In [15]:
P[...,1,0].real.mean(), P[...,1,0].real.std(),P[...,1,0].imag.mean(), P[...,1,0].imag.std()

(tensor(-0.0014), tensor(0.4994), tensor(0.0052), tensor(0.5030))

In [16]:
P[...,0,0].real.mean(), P[...,0,0].real.std(),P[...,0,0].imag.mean(), P[...,0,0].imag.std()

(tensor(0.), tensor(0.), tensor(0.0014), tensor(0.5047))

In [17]:
tr.sqrt(tr.tensor(2))

tensor(1.4142)

In [18]:
pp = 2.0*tr.einsum('kij,...ji->...k',s.tau,P)

In [19]:
pp.real.std(dim=(0,1,2))

tensor([1.0061, 0.9988, 1.0094])

In [20]:
pp.shape

torch.Size([2, 128, 128, 3])

In [21]:
ff = -tr.einsum('...k,kij->...ij',pp,s.tau)

In [22]:
ff[0,0,0]-P[0,0,0]

tensor([[0.+0.j, 0.+0.j],
        [0.+0.j, -0.+0.j]])

In [23]:
tr.trace(P[0,0,0])

tensor(0.+0.j)

In [24]:
tr.einsum('...ij,...ij->',P,P.conj()).real

tensor(49622.0703)

In [25]:
tr.einsum('...k,...k->',pp,pp)/2

tensor(49622.0664+0.j)

In [26]:
(tr.norm(P,dim=(su2c.f.ci[0],su2c.f.ci[1])) - tr.norm(pp,dim=(len(pp.shape)-1))/tr.sqrt(tr.tensor(2))).norm()

tensor(1.2493e-05)

In [27]:
E = su2c.f.expo(P)

In [28]:
E[0,0,0] @ E[0,0,0].conj().transpose(0,1)

tensor([[1.0000e+00+0.0000e+00j, 2.4200e-09-2.9802e-08j],
        [2.4200e-09+2.9802e-08j, 1.0000e+00+0.0000e+00j]])

In [29]:
tr.eye(2,2) + P[0,0,0]@(tr.eye(2,2) + 0.5*P[0,0,0]@(tr.eye(2,2)+ 1/3*P[0,0,0]@(tr.eye(2,2) + 1/4*P[0,0,0]))), E[0,0,0]

(tensor([[ 0.8774-0.1190j, -0.2361+0.4001j],
         [ 0.2361+0.4001j,  0.8774+0.1190j]]),
 tensor([[ 0.8774-0.1190j, -0.2362+0.4003j],
         [ 0.2362+0.4003j,  0.8774+0.1190j]]))